In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ipynb.fs.full.NBAStatsWebScraper import load_NBA_stats

In [25]:
df_NBA_complete = load_NBA_stats().drop(['Tm'], axis=1)
target_col = 'Pos'
df_NBA_complete.head()

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2022/23 Salary
0,Precious Achiuwa,C,23,44,11,22.0,3.6,7.5,0.476,0.5,1.9,0.256,3.1,5.7,0.548,0.508,1.7,2.5,0.691,1.9,4.3,6.1,1.0,0.6,0.6,1.2,2.0,9.4,2840160
1,Steven Adams,C,29,42,42,27.0,3.7,6.3,0.597,0.0,0.0,0.000,3.7,6.2,0.599,0.597,1.1,3.1,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6,17926829
2,Bam Adebayo,C,25,64,64,35.1,8.3,15.4,0.541,0.0,0.2,0.083,8.3,15.2,0.547,0.542,4.4,5.4,0.807,2.4,7.0,9.4,3.3,1.2,0.8,2.6,2.9,21.0,30351780
3,Ochai Agbaji,SG,22,45,8,17.3,2.0,4.6,0.440,1.1,3.0,0.366,1.0,1.7,0.573,0.557,0.5,0.7,0.742,0.6,1.1,1.8,0.7,0.2,0.2,0.4,1.5,5.7,3918360
4,Santiago Aldama,PF,22,63,18,22.1,3.3,7.0,0.475,1.3,3.7,0.359,2.0,3.4,0.601,0.569,1.3,1.8,0.719,1.0,3.6,4.6,1.2,0.6,0.7,0.7,1.9,9.3,2094120


In [26]:
dict_imputation = dict()
for col in df_NBA_complete.columns:
    if col == target_col or col == 'Player':
        continue
    dict_imputation[col] = round(df_NBA_complete[col].mean(),3)
    df_NBA_complete[col].fillna(dict_imputation[col],inplace=True)

In [27]:
dict_imputation

{'Age': 25.75,
 'G': 41.188,
 'GS': 19.548,
 'MP': 19.49,
 'FG': 3.283,
 'FGA': 6.99,
 'FG%': 0.464,
 '3P': 0.969,
 '3PA': 2.727,
 '3P%': 0.321,
 '2P': 2.316,
 '2PA': 4.262,
 '2P%': 0.535,
 'eFG%': 0.532,
 'FT': 1.436,
 'FTA': 1.844,
 'FT%': 0.75,
 'ORB': 0.858,
 'DRB': 2.632,
 'TRB': 3.49,
 'AST': 1.995,
 'STL': 0.601,
 'BLK': 0.374,
 'TOV': 1.095,
 'PF': 1.679,
 'PTS': 8.966,
 '2022/23 Salary': 8611931.237}

In [28]:
dict_normalize = dict()
for col in df_NBA_complete.columns:
    if col == target_col or col == 'Player':
        continue
    dict_normalize[col] = (df_NBA_complete[col].min(),df_NBA_complete[col].max())
    df_NBA_complete[col] = df_NBA_complete[col].apply(lambda x : (x - dict_normalize[col][0])/(dict_normalize[col][1] - dict_normalize[col][0]))
dict_normalize

{'Age': (19, 42),
 'G': (1, 69),
 'GS': (0, 69),
 'MP': (1.0, 37.5),
 'FG': (0.0, 11.3),
 'FGA': (0.0, 22.6),
 'FG%': (0.0, 1.0),
 '3P': (0.0, 5.0),
 '3PA': (0.0, 11.5),
 '3P%': (0.0, 1.0),
 '2P': (0.0, 10.2),
 '2PA': (0.0, 17.7),
 '2P%': (0.0, 1.0),
 'eFG%': (0.0, 1.0),
 'FT': (0.0, 10.2),
 'FTA': (0.0, 12.7),
 'FT%': (0.0, 1.0),
 'ORB': (0.0, 5.1),
 'DRB': (0.0, 9.7),
 'TRB': (0.0, 12.4),
 'AST': (0.0, 10.8),
 'STL': (0.0, 2.2),
 'BLK': (0.0, 3.2),
 'TOV': (0.0, 4.1),
 'PF': (0.0, 3.9),
 'PTS': (0.0, 33.4),
 '2022/23 Salary': (58493, 48070014)}

In [29]:
df_NBA_complete

,Player,Pos,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,2022/23 Salary
0,Precious Achiuwa,C,0.173913,0.632353,0.159420,0.575342,0.318584,0.331858,0.476,0.10,0.165217,0.256,0.303922,0.322034,0.548,0.508,0.166667,0.196850,0.691,0.372549,0.443299,0.491935,0.092593,0.272727,0.18750,0.292683,0.512821,0.281437,0.057937
1,Steven Adams,C,0.434783,0.602941,0.608696,0.712329,0.327434,0.278761,0.597,0.00,0.000000,0.000,0.362745,0.350282,0.599,0.597,0.107843,0.244094,0.364,1.000000,0.670103,0.927419,0.212963,0.409091,0.34375,0.463415,0.589744,0.257485,0.372168
2,Bam Adebayo,C,0.260870,0.926471,0.927536,0.934247,0.734513,0.681416,0.541,0.00,0.017391,0.083,0.813725,0.858757,0.547,0.542,0.431373,0.425197,0.807,0.470588,0.721649,0.758065,0.305556,0.545455,0.25000,0.634146,0.743590,0.628743,0.630959
3,Ochai Agbaji,SG,0.130435,0.647059,0.115942,0.446575,0.176991,0.203540,0.440,0.22,0.260870,0.366,0.098039,0.096045,0.573,0.557,0.049020,0.055118,0.742,0.117647,0.113402,0.145161,0.064815,0.090909,0.06250,0.097561,0.384615,0.170659,0.080395
4,Santiago Aldama,PF,0.130435,0.911765,0.260870,0.578082,0.292035,0.309735,0.475,0.26,0.321739,0.359,0.196078,0.192090,0.601,0.569,0.127451,0.141732,0.719,0.196078,0.371134,0.370968,0.111111,0.272727,0.21875,0.170732,0.487179,0.278443,0.042399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Thaddeus Young,PF,0.652174,0.779412,0.130435,0.375342,0.176991,0.163717,0.545,0.02,0.052174,0.176,0.186275,0.169492,0.622,0.561,0.029412,0.039370,0.692,0.254902,0.185567,0.250000,0.129630,0.454545,0.03125,0.195122,0.410256,0.131737,0.165408
511,Trae Young,PG,0.217391,0.882353,0.884058,0.931507,0.752212,0.867257,0.433,0.44,0.573913,0.337,0.617647,0.734463,0.482,0.490,0.745098,0.677165,0.889,0.156863,0.226804,0.241935,0.935185,0.500000,0.03125,1.000000,0.384615,0.802395,0.771440
512,Omer Yurtseven,C,0.217391,0.000000,0.000000,0.164384,0.000000,0.044248,0.000,0.00,0.000000,0.321,0.000000,0.056497,0.000,0.000,0.000000,0.000000,0.750,0.196078,0.000000,0.080645,0.000000,0.000000,0.00000,0.000000,0.256410,0.000000,0.035286
513,Cody Zeller,C,0.478261,0.132353,0.000000,0.328767,0.132743,0.128319,0.517,0.00,0.008696,0.000,0.147059,0.158192,0.536,0.517,0.166667,0.181102,0.739,0.313725,0.195876,0.282258,0.055556,0.090909,0.12500,0.195122,0.589744,0.140719,0.014276


In [30]:
def knn_class(df_train, k, target_col, observation, use_weighted_vote ):
   validation = df_train[target_col].copy()
   train = df_train.drop([target_col], axis=1)
   observed_clean = observation.drop([target_col])
   norms = np.linalg.norm((train-observed_clean).astype(float), axis=1)
   idx = np.argpartition(norms,k)[:k]
   if use_weighted_vote:
      weighted_sum = {key: 0 for key in validation.value_counts().index}
      weights = 1/(norms**2)
      for i in idx:
         weighted_sum[validation[i]] += weights[i]
      return max(weighted_sum, key=weighted_sum.get)
   nearest_types = validation.take(idx)
   return nearest_types.mode()[0]